# Lesson-04

**Написать приложение, которое собирает основные новости с сайтов news.mail.ru, lenta.ru, yandex.news. Для парсинга использовать xpath. Структура данных должна содержать:**

**1. Название источника**

**2. Наименование новости**

**3. Ссылку на новость**

**4. Дата публикации**

In [26]:
from lxml import html
from requests import get
from pprint import pprint
import re
from datetime import datetime as dt
from pymongo import MongoClient

In [27]:
header = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36"}

**lenta.ru**

In [28]:
def request_to_lenta():
    lenta_link = 'https://lenta.ru/'
    response = get(lenta_link,
                 headers = header
                 )
    
    root = html.fromstring(response.text)
    result = []
    items = root.xpath("//section[@class='row b-top7-for-main js-top-seven']//div[@class='item'] | //section[@class='row b-top7-for-main js-top-seven']//div[@class='first-item']")

    for item in items:
        info = {}
        name_source = 'https://lenta.ru/'
        name_news = item.xpath(".//a[not(@class)]/text() | .//a[@class='b-link-external']/text()")[0]
        name_news = name_news.replace('\xa0', ' ')
        link = item.xpath(".//a[not(@class)]/@href | .//a[@class='b-link-external']/@href")[0]
        date = item.xpath(".//a[not(@class)]//@datetime | .//a[@class='b-link-external']//@datetime")[0]
        date = re.findall(r'\d+\:\d+', date)[0]
        date = dt.strptime(date+ ' ' + str(dt.today().date()), '%H:%M %Y-%m-%d')

        info['source'] = name_source
        info['name'] = name_news
        info['link'] = lenta_link+link
        info['datetime'] = date
        result.append(info)
    return result

**yandex.news**

In [29]:
def request_to_yandex():
    yandex_link = 'https://yandex.ru'
    response = get(yandex_link + '/news',
                 headers = header
                 )
    
    root = html.fromstring(response.text)
    result = []
    items = root.xpath("//div[@class='page-content__fixed page-content__fixed_middle']/div/table//td")
    
    for item in items:
        info = {}
        name_source = item.xpath(".//div[@class='story__info']/div[@class='story__date']/text()")[0]
        name_news = item.xpath(".//h2/a/text()")[0]
        link = item.xpath(".//h2/a//@href")[0]
        date = re.findall(r'\d+\:\d+', name_source)[0]
        name_source = name_source.replace(' ' + date, '')
        try:
            name_source = name_source.replace(re.findall(r' \w+\xa0\w\xa0\d+\:\d+', name_source)[0], '')
        except:
            name_source = name_source
            
        date = dt.strptime(date+ ' ' + str(dt.today().date()), '%H:%M %Y-%m-%d')
        
        
        info['source'] = name_source
        info['name'] = name_news
        info['link'] = yandex_link + link
        info['datetime'] = date
        result.append(info)
    return result

**news.mail.ru**

In [30]:
def request_to_mail():
    mail_link = 'https://news.mail.ru/'
    response = get(mail_link,
                 headers = header
                 )
    
    root = html.fromstring(response.text)
    result = []
    items = root.xpath("//div[@class='js-module']/div//td/div[@class='daynews__item daynews__item_big'] | //div[@class='js-module']/div//td/div[@class='daynews__item'] | //div[@class='js-module']/ul/li[@class='list__item'] | //div[@class='js-module']/ul/li[@class='list__item hidden_small']")
    
    for item in items:
        info = {}
        name_source = item.xpath(".//a/@href")[0]
        

        response_sub = get(mail_link + name_source,
                    headers = header
                    )
        if response_sub.ok:
            info['link'] = mail_link + name_source
            root_sub = html.fromstring(response_sub.text)
            result_sub = []
            date = root_sub.xpath("//div[@class='article js-article js-module']/div/span//@datetime")[0]
            date = re.findall(r'\d+\-\d+\-\w+\:\d+', date)[0]
            date = date.replace('T', ' ')
            date = dt.strptime(date, '%Y-%m-%d %H:%M')
            info['datetime'] = date
            name = root_sub.xpath("//div[@class='article js-article js-module']/div//h1/text()")[0]
            info['name'] = name
            source = root_sub.xpath("//span[@class='breadcrumbs__item']//span[@class='link__text']/text()")[0]
            info['source'] = source
            
        else:
            response_sub = get(name_source,
                            headers = header
                             ) 
            root_sub = html.fromstring(response_sub.text)
            info['link'] = name_source
            result_sub = []
            date = root_sub.xpath("//div[@class='article js-article']//span//@datetime")[0]
            date = re.findall(r'\d+\-\d+\-\w+\:\d+', date)[0]
            date = date.replace('T', ' ')
            date = dt.strptime(date, '%Y-%m-%d %H:%M')
            info['datetime'] = date
            name = root_sub.xpath("//h1/text()")[0]
            info['name'] = name
            source = root_sub.xpath("//span[@class='breadcrumbs__item']//span[@class='link__text']/text()")[0]
            info['source'] = source
        
        result.append(info)
    return result

**Сложить все новости в БД**

In [31]:
lenta = request_to_lenta()
pprint(lenta)

[{'datetime': datetime.datetime(2020, 4, 20, 10, 35),
  'link': 'https://lenta.ru//news/2020/04/20/covid/',
  'name': 'Еще 4268 случаев заражения коронавирусом выявлено в России за сутки',
  'source': 'https://lenta.ru/'},
 {'datetime': datetime.datetime(2020, 4, 20, 11, 7),
  'link': 'https://lenta.ru//news/2020/04/20/krym/',
  'name': 'Россиян призвали отказаться от поездок в Крым',
  'source': 'https://lenta.ru/'},
 {'datetime': datetime.datetime(2020, 4, 20, 11, 3),
  'link': 'https://lenta.ru//news/2020/04/20/love/',
  'name': 'Медбрат позвал замуж коллегу после двух месяцев работы в '
          'коронавирусной больнице',
  'source': 'https://lenta.ru/'},
 {'datetime': datetime.datetime(2020, 4, 20, 11, 1),
  'link': 'https://lenta.ru/https://moslenta.ru/news/v-moskve-ocenili-rabotu-propusknogo-rezhima-na-dorogakh-20-04-2020.htm/?utm_source=from_lenta',
  'name': 'В Москве оценили работу пропускного режима на дорогах ',
  'source': 'https://lenta.ru/'},
 {'datetime': datetime.date

In [32]:
yandex = request_to_yandex()
pprint(yandex)

[{'datetime': datetime.datetime(2020, 4, 20, 10, 57),
  'link': 'https://yandex.ru/news/story/Obshhezhitie_Pervogo_medicinskogo_universiteta_zakryli_na_karantin--bdcf28908170c2c7900756fa97990f81?lr=2&lang=ru&stid=-1Ewqr76FkXOYtO7Inl6&persistent_id=94962877&rubric=Saint_Petersburg&from=index',
  'name': 'Общежитие Первого медицинского университета закрыли на карантин',
  'source': 'Карповка'},
 {'datetime': datetime.datetime(2020, 4, 20, 10, 49),
  'link': 'https://yandex.ru/news/story/Situaciya_s_koronavirusom_v_Peterburge_otstaet_ot_stolicy_na_dve_nedeli--712b8957b3f010eaa02c3e826a5e0cfb?lr=2&lang=ru&stid=GxBfwmv7sTeJ_pnXb4Se&persistent_id=94941206&rubric=Saint_Petersburg&from=index',
  'name': 'Ситуация с коронавирусом в Петербурге отстает от столицы на две '
          'недели',
  'source': 'Piter.tv'},
 {'datetime': datetime.datetime(2020, 4, 20, 11, 0),
  'link': 'https://yandex.ru/news/story/Indeks_samoizolyacii_rezko_upal_v_Pskove--2b190175a5a4d9eba5f75f109bbcda14?lr=2&lang=ru&st

In [33]:
mail = request_to_mail()
pprint(mail)

[{'datetime': datetime.datetime(2020, 4, 20, 11, 2),
  'link': 'https://news.mail.ru//society/41460513/',
  'name': 'В России число заболевших коронавирусом превысило 47 тысяч человек',
  'source': 'Интерфакс'},
 {'datetime': datetime.datetime(2020, 4, 20, 9, 58),
  'link': 'https://news.mail.ru//society/41458830/',
  'name': 'Треть россиян поддерживают возможный перенос майских праздников',
  'source': 'Коммерсантъ'},
 {'datetime': datetime.datetime(2020, 4, 20, 0, 42),
  'link': 'https://news.mail.ru//society/41455831/',
  'name': 'Названы российские регионы с приростом населения',
  'source': 'РИА Новости'},
 {'datetime': datetime.datetime(2020, 4, 20, 10, 0),
  'link': 'https://news.mail.ru//society/41458601/',
  'name': 'В Москве сократилось число поступлений тяжелых пациентов с '
          'коронавирусом',
  'source': 'ТАСС'},
 {'datetime': datetime.datetime(2020, 4, 20, 10, 21),
  'link': 'https://news.mail.ru//incident/41458627/',
  'name': 'В МВД рассказали, что мошенники все 

In [34]:
client = MongoClient('localhost', 27017)
db = client['news']

In [35]:
def insert_news(site):
    for i in site:
        db.news.update_one({'link': i['link']}, 
                                     {'$set': i},
                                      upsert=True
                                      )

In [36]:
insert_news(lenta)
insert_news(yandex)
insert_news(mail)

In [37]:
objects = db.news.find( {} )
for obj in objects:
    pprint(obj)

{'_id': ObjectId('5e9d59d5794321e75f07e914'),
 'datetime': datetime.datetime(2020, 4, 20, 10, 35),
 'link': 'https://lenta.ru//news/2020/04/20/covid/',
 'name': 'Еще 4268 случаев заражения коронавирусом выявлено в России за сутки',
 'source': 'https://lenta.ru/'}
{'_id': ObjectId('5e9d59d5794321e75f07e916'),
 'datetime': datetime.datetime(2020, 4, 20, 11, 7),
 'link': 'https://lenta.ru//news/2020/04/20/krym/',
 'name': 'Россиян призвали отказаться от поездок в Крым',
 'source': 'https://lenta.ru/'}
{'_id': ObjectId('5e9d59d5794321e75f07e918'),
 'datetime': datetime.datetime(2020, 4, 20, 11, 3),
 'link': 'https://lenta.ru//news/2020/04/20/love/',
 'name': 'Медбрат позвал замуж коллегу после двух месяцев работы в '
         'коронавирусной больнице',
 'source': 'https://lenta.ru/'}
{'_id': ObjectId('5e9d59d5794321e75f07e91a'),
 'datetime': datetime.datetime(2020, 4, 20, 11, 1),
 'link': 'https://lenta.ru/https://moslenta.ru/news/v-moskve-ocenili-rabotu-propusknogo-rezhima-na-dorogakh-20-